# **SISTEMA DE INFORMACIÓN SOBRE LA OCUPACIÓN DE BUSES DEL TRANSPORTE PUBLICO.**


El proyecto se enfoca en el desarrollo de un dispositivo inteligente, capaz de determinar el grado de ocupación de los buses de transporte público mediante el uso de técnicas de inteligencia artificial. 
<p align="center">
  <img src="https://gitlab.com/IsRaTiAl/v4c/-/raw/master/videos/gifs/track+countv1.gif">
</p>

Primero configuramos TF versión 1 e instalamos `filterpy`



In [1]:
%tensorflow_version 1.x
!pip install filterpy

TensorFlow 1.x selected.
     |████████████████████████████████| 184kB 4.8MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-cp36-none-any.whl size=110451 sha256=8ed2bfecd59018e28cb0b250b5c2b2ef109fcb7a66cfb49ae9e8d5483c88e1ba
  Stored in directory: /root/.cache/pip/wheels/c3/0c/dd/e92392c3f38a41371602d99fc77d6c1d42aadbf0c6afccdd02
Successfully built filterpy


Descargamos [Darkflow](https://github.com/thtrieu/darkflow://) de la implementación de YOLO y los pesos de la versión 2 

In [2]:
!git clone https://github.com/thtrieu/darkflow.git
%cd darkflow
!python setup.py build_ext --inplace
!pip install -e .
!pip install .
!mkdir bin
!wget  https://pjreddie.com/media/files/yolov2.weights -O bin/yolov2.weights
%cd ..

Cloning into 'darkflow'...
remote: Enumerating objects: 2713, done.
remote: Total 2713 (delta 0), reused 0 (delta 0), pack-reused 2713
Receiving objects: 100% (2713/2713), 32.98 MiB | 27.66 MiB/s, done.
Resolving deltas: 100% (1762/1762), done.
/content/darkflow
Compiling darkflow/cython_utils/nms.pyx because it changed.
Compiling darkflow/cython_utils/cy_yolo2_findboxes.pyx because it changed.
Compiling darkflow/cython_utils/cy_yolo_findboxes.pyx because it changed.
[1/3] Cythonizing darkflow/cython_utils/cy_yolo2_findboxes.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/darkflow/darkflow/cython_utils/cy_yolo2_findboxes.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[2/3] Cythonizing darkflow/cython_utils/cy_yolo_findboxes.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython direct

Creamos una copia de nuestro repositorio donde se encuentran algunas librerías necesarias y el People couting dataset [PCDS](https://freesoft.dev/program/128588362).

In [3]:
!git clone https://gitlab.com/IsRaTiAl/v4c.git
!cp v4c/notebooks/sort.py darkflow/sort.py
!cp -r v4c/videos/test darkflow/test_videos

Cloning into 'v4c'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 145 (delta 58), reused 119 (delta 42), pack-reused 0
Receiving objects: 100% (145/145), 288.74 MiB | 33.69 MiB/s, done.
Resolving deltas: 100% (58/58), done.


Generamos algunos directorios que los emplearemos más adelante

In [4]:
%cd darkflow/
!mkdir test_videos/front test_videos/front/crowd test_videos/front/uncrowd 
!mkdir test_videos/back test_videos/back/crowd test_videos/back/uncrowd 

!unrar e test_videos/crowd_day_back.rar test_videos/back/crowd
!unrar e test_videos/uncrowd_day_back.rar test_videos/back/uncrowd
!unrar e test_videos/crowd_day_front.rar test_videos/front/crowd
!unrar e test_videos/uncrowd_day_front.rar test_videos/front/uncrowd

!mkdir outputs/
!mkdir outputs/front outputs/front/crowd outputs/front/uncrowd 
!mkdir outputs/back outputs/back/crowd outputs/back/uncrowd

/content/darkflow

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from test_videos/crowd_day_back.rar

Extracting  test_videos/back/crowd/2016_04_07_08_23_45BackColor.avi_red.avi      10% 13%  OK 
Extracting  test_videos/back/crowd/2016_04_07_08_02_36BackColor.avi_red.avi      24% 28%  OK 
Extracting  test_videos/back/crowd/2016_04_07_07_43_03BackColor.avi_red.avi      35%  OK 
Extracting  test_videos/back/crowd/2016_04_07_07_26_50BackColor.avi_red.avi      42%  OK 
Extracting  test_videos/back/crowd/2016_04_07_08_29_20BackColor.avi_red.avi      51%  OK 
Extracting  test_videos/back/crowd/2016_04_07_07_11_10BackColor.avi_red.avi      58%  OK 
Extracting  test_videos/back/crowd/2016_04_07_07_22_10BackColor.avi_red.avi      67%  OK 
Extracting  test_videos/back/crowd/2016_04_07_08_16_23BackColor.avi_red.avi      77%  OK 
Extracting  test_videos/back/crowd/2016_04_07_07_19_24Bac

In [0]:
# !python flow --model cfg/yolo.cfg --load bin/yolov2.weights --demo 2015_05_08_08_27_20BackColor.avi --gpu 1.0 --saveVideo

Importamos algunas las librerias `cv2`, `matplotlib`, `numpy` y `os` para la vision artificial, manejo de graficas, matrices y archivos respectivamente. Adicionalmente, importamos `TFNet` y `sort` para la detección y seguimiento de las personas.

In [6]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from darkflow.net.build import TFNet
from sort import *

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.











/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


Instanciamos el objeto tfnet y le pasamos los parámetros  options, donde se especifican el modelo, los pesos, el umbral de detección y habilitamos el uso de la GPU.

In [7]:
options = {"model": "cfg/yolo.cfg", "load": "bin/yolov2.weights", "threshold": 0.20, 'gpu':1}
tfnet = TFNet(options)

# imgcv = cv2.imread("f2.jpg")
# result = tfnet.return_predict(imgcv)
# print(result)

Parsing cfg/yolo.cfg
Loading bin/yolov2.weights ...
Successfully identified 203934260 bytes
Finished in 0.015497684478759766s
Model has a coco model name, loading coco labels.

Building net ...

Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------



       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)



/content/darkflow/darkflow/dark/darknet.py:54: UserWarning: ./cfg/yolov2.cfg not found, use cfg/yolo.cfg instead
  cfg_path, FLAGS.model))


 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 76, 76, 256)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 76, 76, 128)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 76, 76, 256)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 38, 38, 256)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 38, 38, 512)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 38, 38, 256)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 38, 38, 512)
 Load  |  Ye

Definimos las funciones:
- `bounbox`: la cual nos permiten generar los recuadros que denotan la deteccion.
- `intersect` y `ccw`: las cuales nos permiter detectar cuando una persona cruza un determinado umbral.

In [0]:
def bounbox(imgcv, box):
  for box in result:
    if(box['label']=='person'):
      x1,y1,x2,y2 = box['topleft']['x'],box['topleft']['y'],box['bottomright']['x'],box['bottomright']['y']
      cx, cy = x1+int((x2-x1)/2), y1+int((y2-y1)/2)
      conf = box['confidence']
      label = box['label']
      cv2.rectangle(imgcv,(x1,y1),(x2,y2),(0,255,0),2)
      labelSize=cv2.getTextSize(label,cv2.FONT_HERSHEY_COMPLEX,0.5,1)
      confSize=cv2.getTextSize(label,cv2.FONT_HERSHEY_COMPLEX,0.5,1)
      _x1 = x1
      _y1 = y1#+int(labelSize[0][1]/2)
      _x2 = _x1+labelSize[0][0]
      _y2 = y1-int(labelSize[0][1])
      cv2.rectangle(imgcv,(_x1,_y1),(_x2,_y2),(0,255,0),cv2.FILLED)
      cv2.putText(imgcv,label,(x1,y1),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,0),1)
      cv2.rectangle(imgcv,(_x1,_y1+14),(_x2,_y2+14),(0,255,0),cv2.FILLED)
      cv2.putText(imgcv,str(round(conf,2)),(x1,y1+14),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,0),1)
      cv2.circle(imgcv,(cx,cy), 5, (0,0,255), -1)
  return imgcv

def intersect(A,B,C,D):
	return ccw(A,C,D) != ccw(B,C,D) and ccw(A,B,C) != ccw(A,B,D)
def ccw(A,B,C):
  return (C[1]-A[1]) * (B[0]-A[0])> (B[1]-A[1]) * (C[0]-A[0])

In [0]:
directory = "/content/darkflow/test_videos/"
output_directory = "/content/darkflow/outputs/"
ubication = 'back/'
amount    = 'uncrowd/'
paths = os.listdir(directory+ubication+amount)

line = [(0, 100), (320, 100)]
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(200, 3),	dtype="uint8")

In [19]:
for path in paths:
  print(path)
  mot_tracker = Sort()
  # capture = cv2.VideoCapture('2016_04_07_08_15_14BackColor.avi')
  capture = cv2.VideoCapture(directory+ubication+amount+path)

  ret, frame = capture.read()
  frame_height,frame_width,_ = frame.shape
  fourcc = cv2.VideoWriter_fourcc(*'XVID')
  out = cv2.VideoWriter(output_directory+ubication+amount+path,fourcc, 20, (frame_width,frame_height))

  mem = {}
  ret = True
  count = 0
  counter = 0
  previous = {}
  while(ret):
    ret, frame = capture.read()
    if(ret):
      # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      result = tfnet.return_predict(frame)

      dets = []
      if len(result) > 0:
          # loop over the indexes we are keeping
          for res in result:
            if(res['label']=='person'):
              (x1,y1)=(res['topleft']['x'], res['topleft']['y'])
              (x2,y2)=(res['bottomright']['x'], res['bottomright']['y'])
              dets.append([x1, y1, x2, y2, res['confidence']])
      dets = np.asarray(dets)
      tracks = mot_tracker.update(dets)

      boxes = []
      indexIDs = []
      c = []
      previous = mem.copy()
      mem = {}
      for track in tracks:
        boxes.append([track[0], track[1], track[2], track[3]])
        indexIDs.append(int(track[4]))
        mem[indexIDs[-1]] = boxes[-1]

      if len(boxes) > 0:
        i = int(0)
        for box in boxes:
          # extract the bounding box coordinates
          (x, y) = (int(box[0]), int(box[1]))
          (w, h) = (int(box[2]), int(box[3]))

          color = [int(c) for c in COLORS[indexIDs[i] % len(COLORS)]]
          cv2.rectangle(frame, (x, y), (w, h), color, 2)
          # print(indexIDs)
          if indexIDs[i] in previous:
            previous_box = previous[indexIDs[i]]
            (x2, y2) = (int(previous_box[0]), int(previous_box[1]))
            (w2, h2) = (int(previous_box[2]), int(previous_box[3]))
            p0 = (int(x + (w-x)/2), int(y + (h-y)/2))
            p1 = (int(x2 + (w2-x2)/2), int(y2 + (h2-y2)/2))
            cv2.line(frame, p0, p1, color, 10)
            if intersect(p0, p1, line[0], line[1]):
              # print(i)
              # print(np.sum(np.square(np.array(p1)-np.array(p0))))
              # print(indexIDs[i])
              # if(np.sum(np.square(np.array(p1)-np.array(p0)))>25):
                counter += 1

          # text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
          # text = "{}".format(indexIDs[i])
          # cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
          i += 1

      # draw line
      cv2.line(frame, line[0], line[1], (0, 255, 255), 2)

      # draw counter
      cv2.putText(frame, str(counter), (300,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
      # counter += 1


      # if(len(result)>0):
        # frame = bounbox(frame,result)
      out.write(frame)
      # if(count%10==0):
      #   plt.imshow(frame)
      #   plt.show()
      count+=1

    # if(count>500):
      # ret = False

  capture.release()
  out.release()

2016_04_07_08_47_18FrontColor_red.avi


/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


2016_04_07_08_48_35FrontColor_red.avi
2016_04_07_07_13_38FrontColor_red.avi
2016_04_07_07_09_58FrontColor_red.avi
2016_04_07_08_45_44FrontColor_red.avi
2016_04_07_14_37_58FrontColor_red.avi
2016_04_07_16_06_09FrontColor_red.avi
2016_04_07_08_01_11FrontColor_red.avi
2016_04_07_07_32_42FrontColor_red.avi
2016_04_07_16_03_49FrontColor_red.avi


In [23]:
for path in paths:
  print(path)
  mot_tracker = Sort()
  # capture = cv2.VideoCapture('2016_04_07_08_15_14BackColor.avi')
  capture = cv2.VideoCapture(directory+ubication+amount+path)

  ret, frame = capture.read()
  frame_height,frame_width,_ = frame.shape
  fourcc = cv2.VideoWriter_fourcc(*'XVID')
  out = cv2.VideoWriter(output_directory+ubication+amount+path,fourcc, 20, (frame_width,frame_height))

  mem = {}
  ret = True
  count = 0
  counter = 0
  disponible = 20
  previous = {}
  while(ret):
    ret, frame = capture.read()
    if(ret):
      # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      result = tfnet.return_predict(frame)

      dets = []
      if len(result) > 0:
          # loop over the indexes we are keeping
          for res in result:
            if(res['label']=='person'):
              (x1,y1)=(res['topleft']['x'], res['topleft']['y'])
              (x2,y2)=(res['bottomright']['x'], res['bottomright']['y'])
              dets.append([x1, y1, x2, y2, res['confidence']])
      dets = np.asarray(dets)
      tracks = mot_tracker.update(dets)

      boxes = []
      indexIDs = []
      c = []
      previous = mem.copy()
      mem = {}
      for track in tracks:
        boxes.append([track[0], track[1], track[2], track[3]])
        indexIDs.append(int(track[4]))
        mem[indexIDs[-1]] = boxes[-1]

      if len(boxes) > 0:
        i = int(0)
        for box in boxes:
          # extract the bounding box coordinates
          (x, y) = (int(box[0]), int(box[1]))
          (w, h) = (int(box[2]), int(box[3]))

          color = [int(c) for c in COLORS[indexIDs[i] % len(COLORS)]]
          cv2.rectangle(frame, (x, y), (w, h), color, 2)
          # print(indexIDs)
          if indexIDs[i] in previous:
            previous_box = previous[indexIDs[i]]
            (x2, y2) = (int(previous_box[0]), int(previous_box[1]))
            (w2, h2) = (int(previous_box[2]), int(previous_box[3]))
            p0 = (int(x + (w-x)/2), int(y + (h-y)/2))
            p1 = (int(x2 + (w2-x2)/2), int(y2 + (h2-y2)/2))
            cv2.line(frame, p0, p1, color, 10)
            if intersect(p0, p1, line[0], line[1]):
              # print(i)
              # print(np.sum(np.square(np.array(p1)-np.array(p0))))
              # print(indexIDs[i])
              # if(np.sum(np.square(np.array(p1)-np.array(p0)))>25):
                counter += 1
                if ubication == 'back/':
                    disponible+= 1
                if ubication == 'front/':
                    disponible-= 1

          # text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
          # text = "{}".format(indexIDs[i])
          # cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
          i += 1

      # draw line
      cv2.line(frame, line[0], line[1], (0, 255, 0), 2)

      # draw counter
      if ubication == 'back/':
          cv2.putText(frame, "Salidas: "+str(counter), (210,200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
          cv2.putText(frame, "Sitios disponibles: "+str(disponible), (150,225), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
      if ubication == 'front/':
          cv2.putText(frame, "Entradas: "+str(counter), (210,200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
          cv2.putText(frame, "Sitios disponibles: "+str(disponible), (150,225), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
      # counter += 1
      if disponible < 0:
          cv2.putText(frame, "Sin Lugares disponibles", (120,120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)


      # if(len(result)>0):
        # frame = bounbox(frame,result)
      out.write(frame)
      # if(count%10==0):
        # plt.imshow(frame)
        # plt.show()
      count+=1

    # if(count>50):
      # ret = False

  capture.release()
  out.release()

2016_04_07_07_30_45BackColor.avi_red.avi


/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


2016_04_07_07_48_06BackColor.avi_red.avi
2016_04_07_08_15_14BackColor.avi_red.avi
2016_04_07_07_32_39BackColor.avi_red.avi
2016_04_07_08_04_57BackColor.avi_red.avi
2016_04_07_07_17_06BackColor.avi_red.avi
2016_04_07_07_15_47BackColor.avi_red.avi
2016_04_07_08_17_51BackColor.avi_red.avi
2016_04_07_07_13_40BackColor.avi_red.avi
